In [1]:
from PIL import ImageOps
from sklearn.metrics import classification_report, accuracy_score

import os
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import pandas as pd
import cv2
import time
import copy
import math



import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim import lr_scheduler
from torch.autograd import Variable

from tqdm import tqdm, notebook

In [2]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [3]:
data_set = ImageFolder('/kaggle/input/handwrittendata/digit',
                        transform=transforms.Compose([
                            transforms.Resize(28),
                            transforms.Grayscale(num_output_channels=1),
                            transforms.ToTensor(),
                            transforms.Normalize((1, ), (-1, ))],
                        )
                      )
class_names = data_set.classes[:]
print(class_names)
print(len(class_names))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'div', 'equal', 'minus', 'plus', 'times']
15


In [4]:
train_size = int(0.6 * len(data_set))
val_size = int(0.2 * len(data_set))
test_size = len(data_set) - train_size - val_size
train_set, val_set, test_set = random_split(data_set, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)

In [5]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
          
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(64 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, len(class_names)),
        )
          
        for m in self.features.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        for m in self.classifier.children():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        
        return x  
    
    def load(self, model_path):
        self.load_state_dict(torch.load(model_path))
        self.eval()

    def predict(self, images):
        outputs = self(images)
        _, predicted = torch.max(outputs, 1)
        return predicted

In [6]:
model = Net()

optimizer_ft = optim.Adam(model.parameters(), lr=0.003)

criterion = nn.CrossEntropyLoss()

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(train_loader)
    val_batches = len(val_loader)
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        loss_hist = {'train': [], 'val': []}
        acc_hist = {'train': [], 'val': []}
        
        model.train(True)
        
        for i, data in enumerate(train_loader):
   
            print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                pass
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        
        avg_loss = loss_train / len(train_set)
        avg_acc = acc_train / len(train_set)
        
        loss_hist['train'].append(avg_loss)
        acc_hist['train'].append(avg_acc)
        
        model.train(False)
        model.eval()
        with torch.no_grad():    
            for i, data in enumerate(val_loader):

                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)

                inputs, labels = data

                if use_gpu:
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    pass

                outputs = model(inputs)

                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                loss_val += loss.data
                acc_val += torch.sum(preds == labels.data)

                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()

            avg_loss_val = loss_val / len(val_set)
            avg_acc_val = acc_val / len(val_set)
            
            loss_hist['val'].append(avg_loss)
            acc_hist['val'].append(avg_acc)

            print()
            print("Epoch {} result: ".format(epoch+1))
            print("Avg loss (train): {:.4f}".format(avg_loss))
            print("Avg acc (train): {:.4f}".format(avg_acc))
            print("Avg loss (val): {:.4f}".format(avg_loss_val))
            print("Avg acc (val): {:.4f}".format(avg_acc_val))
            print('-' * 10)
            print()

            if avg_acc_val > best_acc:
                best_acc = avg_acc_val
                best_model_wts = copy.deepcopy(model.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model, loss_hist, acc_hist

In [8]:
model, loss_hist, acc_hist = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)
torch.save(model.state_dict(), 'model_v3.pt')

Epoch 0/5
----------
Training batch 1599/1600
Validation batch 533/534
Epoch 1 result: 
Avg loss (train): 0.0023
Avg acc (train): 0.9599
Avg loss (val): 0.0005
Avg acc (val): 0.9924
----------

Epoch 1/5
----------
Training batch 1599/1600
Validation batch 533/534
Epoch 2 result: 
Avg loss (train): 0.0010
Avg acc (train): 0.9841
Avg loss (val): 0.0004
Avg acc (val): 0.9940
----------

Epoch 2/5
----------
Training batch 1599/1600
Validation batch 533/534
Epoch 3 result: 
Avg loss (train): 0.0008
Avg acc (train): 0.9877
Avg loss (val): 0.0003
Avg acc (val): 0.9956
----------

Epoch 3/5
----------
Training batch 1599/1600
Validation batch 533/534
Epoch 4 result: 
Avg loss (train): 0.0006
Avg acc (train): 0.9898
Avg loss (val): 0.0004
Avg acc (val): 0.9948
----------

Epoch 4/5
----------
Training batch 1599/1600
Validation batch 533/534
Epoch 5 result: 
Avg loss (train): 0.0006
Avg acc (train): 0.9910
Avg loss (val): 0.0003
Avg acc (val): 0.9966
----------


Training completed in 31m 31s

In [9]:
y_pred_list = []
y_true_list = []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.cuda(), y_batch.cuda()
        y_test_pred = model(x_batch)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())
        
y_pred_list = [i[0] for i in y_pred_list]
y_true_list = [i[0] for i in y_true_list]

100%|██████████| 534/534 [03:46<00:00,  2.36it/s]


In [10]:
print(classification_report(y_true_list, y_pred_list))
print(confusion_matrix(y_true_list, y_pred_list))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      0.99      0.99        84
           2       1.00      1.00      1.00        82
           3       1.00      1.00      1.00        36
           4       0.95      1.00      0.98        20
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00        10
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00        46
          12       1.00      0.99      1.00       109
          13       1.00      1.00      1.00        88
          14       0.90      1.00      0.95         9

    accuracy                           1.00       534
   macro avg       0.99      1.00      0.99       534
weighted avg       1.00   